## 시각화 있음

In [6]:
import datetime
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from scipy.signal import argrelextrema

CONFIDENCE_THRESHOLD = 0.4
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)

"""
Tracking + ON-OFF 알고리즘 함수
"""
def load_class_list(file_path):
    with open(file_path, 'r') as f:
        return f.read().split('\n')

def initialize_yolo_and_tracker(model_path):
    model = YOLO(model_path)
    tracker = DeepSort(max_age=20)
    return model, tracker

def get_image_files(image_folder_path):
    file_names = os.listdir(image_folder_path)
    image_files = [f for f in file_names if f.endswith(('.jpg', '.png'))]
    image_files.sort()
    return image_files

def initialize_video_writer(output_video_path, frame_width, frame_height):
    return cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (frame_width, frame_height))

def get_extrema_diff_threshold(r_value):
    return 20

def analyze_cycle_fft(r_values):
    fft_values = np.fft.fft(r_values)
    frequencies = np.fft.fftfreq(len(r_values))
    return frequencies, np.abs(fft_values)

def calculate_shift_mean(r_values, window_size=2):
    if len(r_values) < window_size:
        return r_values
    return np.convolve(r_values, np.ones(window_size) / window_size, mode='valid')

def analyze_cycle_extrema(r_values, extrema_diff_threshold, window_size=2):
    r_shift_mean_values = calculate_shift_mean(r_values, window_size)
    if len(r_shift_mean_values) < 46:
        return False, 0

    max_valid_extrema_count = 0
    for start in range(len(r_shift_mean_values) - 44):
        segment = r_shift_mean_values[start:start + 45]
        maxima = argrelextrema(np.array(segment), np.greater)[0]
        minima = argrelextrema(np.array(segment), np.less)[0]
        extrema = np.sort(np.concatenate((maxima, minima)))

        valid_extrema = []
        for i in range(1, len(extrema)):
            diff = abs(segment[extrema[i]] - segment[extrema[i - 1]])
            if diff > extrema_diff_threshold:
                valid_extrema.append(extrema[i - 1])
                valid_extrema.append(extrema[i])

        valid_extrema = np.unique(valid_extrema)
        max_valid_extrema_count = max(max_valid_extrema_count, len(valid_extrema))

    has_valid_extrema = 3 <= max_valid_extrema_count <= 7
    return has_valid_extrema, max_valid_extrema_count


"""
    그룹화 함수
"""

def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    return inter_area / (box1_area + box2_area - inter_area) if (box1_area + box2_area - inter_area) > 0 else 0

def calculate_containment_ratio(outer_box, inner_box):
    x1_min, y1_min, x1_max, y1_max = outer_box
    x2_min, y2_min, x2_max, y2_max = inner_box

    inner_area = (x2_max - x2_min) * (y2_max - y2_min)
    outer_area = (x1_max - x1_min) * (y1_max - y1_min)

    if outer_area > 0:
        containment_ratio = inner_area / outer_area
        return containment_ratio >= 0.90
    return False

def group_boxes(boxes, labels, confidences, iou_min=0.02, iou_max=0.13):
    groups = []
    used = set()

    for i, box in enumerate(boxes):
        if i in used or labels[i] != 0:
            continue
        group = [(box, confidences[i])]
        group_labels = [labels[i]]
        used.add(i)

        blinkers = []
        for j, other_box in enumerate(boxes):
            if j != i and j not in used and labels[j] == 1:
                iou = calculate_iou(box, other_box)
                if iou_min <= iou <= iou_max or calculate_containment_ratio(box, other_box):
                    blinkers.append((other_box, confidences[j]))
                    used.add(j)

        blinkers = blinkers[:2]
        for blinker_box, blinker_conf in blinkers:
            group.append((blinker_box, blinker_conf))
            group_labels.append(labels[i])

        groups.append((group, group_labels))

    return groups

def update_labels(grouped_boxes):
    updated_boxes = []

    for idx, (group, labels) in enumerate(grouped_boxes):
        car_label = f"car{idx + 1}"

        sorted_group = sorted(group, key=lambda item: item[0][0])

        if len(sorted_group) == 3:
            updated_labels = [f"{car_label}-1", f"{car_label}-L", f"{car_label}-R"]
            updated_boxes.append((sorted_group, updated_labels))
        else:
            updated_labels = [f"{car_label}-{j + 1}" for j in range(len(group))]
            updated_boxes.append((sorted_group, updated_labels))

    return updated_boxes

def process_frame(frame, model, tracker, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count):
    detection = model.predict(source=[frame], save=False)[0]
    results = []

    for data in detection.boxes.data.tolist():
        confidence = float(data[4])
        if confidence < CONFIDENCE_THRESHOLD:
            continue

        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        width = xmax - xmin
        height = ymax - ymin
        label = int(data[5])
        class_name = class_list[label]

        if class_name not in class_track_ids:
            class_track_ids[class_name] = {}

        bbox = (xmin, ymin, width, height)
        results.append([bbox, confidence, label])

    tracks = tracker.update_tracks(results, frame=frame)
    process_tracks(tracks, frame, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count)

def process_tracks(tracks, frame, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count):
    boxes = []
    labels = []
    confidences = []
    valid_extrema_tracks = []

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        label = track.det_class
        class_name = class_list[label]

        if track_id not in class_track_ids[class_name]:
            class_track_ids[class_name][track_id] = f"{class_name}-{len(class_track_ids[class_name]) + 1}"

        assigned_id = class_track_ids[class_name][track_id]
        xmin, ymin, xmax, ymax = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
        label_text = f"{assigned_id}"

        if class_name == "car":
            box_color = GREEN
        elif class_name == "light":
            box_color = WHITE
            if track.is_confirmed() and track.time_since_update == 0:
                light_region = frame[ymin:ymax, xmin:xmax]
                avg_color = cv2.mean(light_region)[:3]
                light_rgb = (int(avg_color[2]), int(avg_color[1]), int(avg_color[0]))
                if not light_rgb_history[assigned_id]:
                    light_rgb_history[assigned_id] = [None] * frame_count
                light_rgb_history[assigned_id].append(light_rgb[0])

            r_values = [r for r in light_rgb_history[assigned_id] if r is not None]
            if len(r_values) > 10:
                extrema_diff_threshold = get_extrema_diff_threshold(r_values[-1])
                frequencies, fft_magnitudes = analyze_cycle_fft(r_values)
                has_valid_extrema, num_extrema = analyze_cycle_extrema(r_values, extrema_diff_threshold=extrema_diff_threshold)
                last_num_extrema_dict[assigned_id] = num_extrema
                if has_valid_extrema:
                    box_color = YELLOW
                    valid_extrema_tracks.append((xmin, ymin, xmax, ymax, label_text))

        # Store box details for grouping
        boxes.append((xmin, ymin, xmax, ymax))
        labels.append(label)
        confidences.append(1.0)

        last_num_extrema = last_num_extrema_dict[assigned_id]
        cv2.putText(frame, f'Peaks: {last_num_extrema}', (int(xmin), int(ymax) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), box_color, 3)
        cv2.putText(frame, label_text, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)


    # Group all tracked boxes for current frame if there are valid extrema tracks
    if valid_extrema_tracks:
        grouped_boxes = group_boxes(boxes, labels, confidences)
        updated_boxes = update_labels(grouped_boxes)

        # Print and display information only for the valid extrema tracks
        for (sorted_group, updated_labels) in updated_boxes:
            for box, updated_label in zip(sorted_group, updated_labels):
                x_min, y_min, x_max, y_max = box[0]
                for v_track in valid_extrema_tracks:
                    if (x_min, y_min, x_max, y_max) == v_track[:4]:
                        print(f"{updated_label} - 방향지시등이 켜졌습니다.")
                        cv2.putText(frame, f"{updated_label} - ON", (int(x_min), int(y_max) + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, RED, 2)


def process_images(image_folder_path, output_video_path, model, tracker, class_list):
    image_files = get_image_files(image_folder_path)
    frame_width, frame_height = cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[1], cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[0]
    out = initialize_video_writer(output_video_path, frame_width, frame_height)
    class_track_ids = {}
    light_rgb_history = defaultdict(list)
    last_num_extrema_dict = defaultdict(lambda: 0)
    frame_count = 0

    for image_file in image_files:
        image_path = os.path.join(image_folder_path, image_file)
        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Warning: Could not load image {image_file}")
            continue

        process_frame(frame, model, tracker, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count)

        cv2.putText(frame, f'Frame: {frame_count}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)
        out.write(frame)
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) == ord('q'):
            break

        frame_count += 1

    out.release()
    cv2.destroyAllWindows()
    
"""
    main 실행코드
"""

def main():
    IMAGE_FOLDER_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/test_data/test4/test4-2'
    OUTPUT_VIDEO_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/output/test4-4_output.mp4'
    MODEL_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/model/best.pt'
    CLASS_LIST_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/model/yolo_class.txt'

    class_list = load_class_list(CLASS_LIST_PATH)
    model, tracker = initialize_yolo_and_tracker(MODEL_PATH)
    process_images(IMAGE_FOLDER_PATH, OUTPUT_VIDEO_PATH, model, tracker, class_list)

if __name__ == "__main__":
    main()



0: 384x640 2 Cars, 2 Lights, 145.1ms
Speed: 1.9ms preprocess, 145.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 2 Lights, 168.8ms
Speed: 1.4ms preprocess, 168.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 2 Lights, 165.8ms
Speed: 1.4ms preprocess, 165.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 2 Lights, 152.6ms
Speed: 1.7ms preprocess, 152.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 2 Lights, 164.1ms
Speed: 1.2ms preprocess, 164.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 2 Lights, 209.6ms
Speed: 1.3ms preprocess, 209.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 3 Lights, 149.5ms
Speed: 1.3ms preprocess, 149.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 2 Lights, 158.8ms
Speed: 1.7ms

KeyboardInterrupt: 

## 시각화 없음

In [10]:
import datetime
import cv2
import os
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from scipy.signal import argrelextrema

CONFIDENCE_THRESHOLD = 0.4
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)

def load_class_list(file_path):
    with open(file_path, 'r') as f:
        return f.read().split('\n')

def initialize_yolo_and_tracker(model_path):
    model = YOLO(model_path)
    tracker = DeepSort(max_age=20)
    return model, tracker

def get_image_files(image_folder_path):
    file_names = os.listdir(image_folder_path)
    image_files = [f for f in file_names if f.endswith(('.jpg', '.png'))]
    image_files.sort()
    return image_files

def initialize_video_writer(output_video_path, frame_width, frame_height):
    return cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (frame_width, frame_height))

def get_extrema_diff_threshold(r_value):
    return 20

def analyze_cycle_fft(r_values):
    fft_values = np.fft.fft(r_values)
    frequencies = np.fft.fftfreq(len(r_values))
    return frequencies, np.abs(fft_values)

def calculate_shift_mean(r_values, window_size=2):
    if len(r_values) < window_size:
        return r_values
    return np.convolve(r_values, np.ones(window_size) / window_size, mode='valid')

def analyze_cycle_extrema(r_values, extrema_diff_threshold, window_size=2):
    r_shift_mean_values = calculate_shift_mean(r_values, window_size)
    if len(r_shift_mean_values) < 46:
        return False, 0

    max_valid_extrema_count = 0
    for start in range(len(r_shift_mean_values) - 44):
        segment = r_shift_mean_values[start:start + 45]
        maxima = argrelextrema(np.array(segment), np.greater)[0]
        minima = argrelextrema(np.array(segment), np.less)[0]
        extrema = np.sort(np.concatenate((maxima, minima)))

        valid_extrema = []
        for i in range(1, len(extrema)):
            diff = abs(segment[extrema[i]] - segment[extrema[i - 1]])
            if diff > extrema_diff_threshold:
                valid_extrema.append(extrema[i - 1])
                valid_extrema.append(extrema[i])

        valid_extrema = np.unique(valid_extrema)
        max_valid_extrema_count = max(max_valid_extrema_count, len(valid_extrema))

    has_valid_extrema = 3 <= max_valid_extrema_count <= 7
    return has_valid_extrema, max_valid_extrema_count

def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    return inter_area / (box1_area + box2_area - inter_area) if (box1_area + box2_area - inter_area) > 0 else 0

def calculate_containment_ratio(outer_box, inner_box):
    x1_min, y1_min, x1_max, y1_max = outer_box
    x2_min, y2_min, x2_max, y2_max = inner_box

    inner_area = (x2_max - x2_min) * (y2_max - y2_min)
    outer_area = (x1_max - x1_min) * (y1_max - y1_min)

    if outer_area > 0:
        containment_ratio = inner_area / outer_area
        return containment_ratio >= 0.90
    return False

def group_boxes(boxes, labels, confidences, iou_min=0.02, iou_max=0.13):
    groups = []
    used = set()

    for i, box in enumerate(boxes):
        if i in used or labels[i] != 0:
            continue
        group = [(box, confidences[i])]
        group_labels = [labels[i]]
        used.add(i)

        blinkers = []
        for j, other_box in enumerate(boxes):
            if j != i and j not in used and labels[j] == 1:
                iou = calculate_iou(box, other_box)
                if iou_min <= iou <= iou_max or calculate_containment_ratio(box, other_box):
                    blinkers.append((other_box, confidences[j]))
                    used.add(j)

        blinkers = blinkers[:2]
        for blinker_box, blinker_conf in blinkers:
            group.append((blinker_box, blinker_conf))
            group_labels.append(labels[i])

        groups.append((group, group_labels))

    return groups

def update_labels(grouped_boxes):
    updated_boxes = []

    for idx, (group, labels) in enumerate(grouped_boxes):
        car_label = f"car{idx + 1}"

        sorted_group = sorted(group, key=lambda item: item[0][0])

        if len(sorted_group) == 3:
            updated_labels = [f"{car_label}-1", f"{car_label}-L", f"{car_label}-R"]
            updated_boxes.append((sorted_group, updated_labels))
        else:
            updated_labels = [f"{car_label}-{j + 1}" for j in range(len(group))]
            updated_boxes.append((sorted_group, updated_labels))

    return updated_boxes

def process_frame(frame, model, tracker, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count):
    # Model prediction
    detection = model.predict(source=[frame], save=False, verbose=False)[0]  # verbose=False to suppress extra YOLO logs
    results = []

    for data in detection.boxes.data.tolist():
        confidence = float(data[4])
        if confidence < CONFIDENCE_THRESHOLD:
            continue

        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        width = xmax - xmin
        height = ymax - ymin
        label = int(data[5])
        class_name = class_list[label]

        if class_name not in class_track_ids:
            class_track_ids[class_name] = {}

        bbox = (xmin, ymin, width, height)
        results.append([bbox, confidence, label])

    tracks = tracker.update_tracks(results, frame=frame)
    process_tracks(tracks, frame, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count)


def process_tracks(tracks, frame, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count):
    boxes = []
    labels = []
    confidences = []
    valid_extrema_tracks = []

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        label = track.det_class
        class_name = class_list[label]

        if track_id not in class_track_ids[class_name]:
            class_track_ids[class_name][track_id] = f"{class_name}-{len(class_track_ids[class_name]) + 1}"

        assigned_id = class_track_ids[class_name][track_id]
        xmin, ymin, xmax, ymax = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
        label_text = f"{assigned_id}"

        if class_name == "car":
            box_color = GREEN
        elif class_name == "light":
            box_color = WHITE
            if track.is_confirmed() and track.time_since_update == 0:
                light_region = frame[ymin:ymax, xmin:xmax]
                avg_color = cv2.mean(light_region)[:3]
                light_rgb = (int(avg_color[2]), int(avg_color[1]), int(avg_color[0]))
                if not light_rgb_history[assigned_id]:
                    light_rgb_history[assigned_id] = [None] * frame_count
                light_rgb_history[assigned_id].append(light_rgb[0])

            r_values = [r for r in light_rgb_history[assigned_id] if r is not None]
            if len(r_values) > 10:
                extrema_diff_threshold = get_extrema_diff_threshold(r_values[-1])
                frequencies, fft_magnitudes = analyze_cycle_fft(r_values)
                has_valid_extrema, num_extrema = analyze_cycle_extrema(r_values, extrema_diff_threshold=extrema_diff_threshold)
                last_num_extrema_dict[assigned_id] = num_extrema
                if has_valid_extrema:
                    box_color = YELLOW
                    valid_extrema_tracks.append((xmin, ymin, xmax, ymax, label_text))

        # Store box details for grouping
        boxes.append((xmin, ymin, xmax, ymax))
        labels.append(label)
        confidences.append(1.0)

        last_num_extrema = last_num_extrema_dict[assigned_id]
        cv2.putText(frame, f'Peaks: {last_num_extrema}', (int(xmin), int(ymax) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), box_color, 3)
        cv2.putText(frame, label_text, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

    # Group all tracked boxes for current frame if there are valid extrema tracks
    if valid_extrema_tracks:
        grouped_boxes = group_boxes(boxes, labels, confidences)
        updated_boxes = update_labels(grouped_boxes)

        # Print information only for the valid extrema tracks
        for (sorted_group, updated_labels) in updated_boxes:
            for box, updated_label in zip(sorted_group, updated_labels):
                x_min, y_min, x_max, y_max = box[0]
                for v_track in valid_extrema_tracks:
                    if (x_min, y_min, x_max, y_max) == v_track[:4]:
                        print(f"프레임 {frame_count} : {updated_label} - 방향지시등이 켜졌습니다.")
    else:
        print(f"프레임 {frame_count} : 속도를 유지하세요.")

def process_images(image_folder_path, output_video_path, model, tracker, class_list):
    image_files = get_image_files(image_folder_path)
    frame_width, frame_height = cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[1], cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[0]
    out = initialize_video_writer(output_video_path, frame_width, frame_height)
    class_track_ids = {}
    light_rgb_history = defaultdict(list)
    last_num_extrema_dict = defaultdict(lambda: 0)
    frame_count = 0

    for image_file in image_files:
        image_path = os.path.join(image_folder_path, image_file)
        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Warning: Could not load image {image_file}")
            continue

        process_frame(frame, model, tracker, class_list, class_track_ids, light_rgb_history, last_num_extrema_dict, frame_count)

        cv2.putText(frame, f'Frame: {frame_count}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)
        out.write(frame)

        frame_count += 1

    out.release()
    cv2.destroyAllWindows()

def main():
    IMAGE_FOLDER_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/test_data/test4/test4-2'
    OUTPUT_VIDEO_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/output/test4-4_output.mp4'
    MODEL_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/model/best.pt'
    CLASS_LIST_PATH = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/model/yolo_class.txt'

    class_list = load_class_list(CLASS_LIST_PATH)
    model, tracker = initialize_yolo_and_tracker(MODEL_PATH)
    process_images(IMAGE_FOLDER_PATH, OUTPUT_VIDEO_PATH, model, tracker, class_list)

if __name__ == "__main__":
    main()


프레임 0 : 속도를 유지하세요.
프레임 1 : 속도를 유지하세요.
프레임 2 : 속도를 유지하세요.
프레임 3 : 속도를 유지하세요.
프레임 4 : 속도를 유지하세요.
프레임 5 : 속도를 유지하세요.
프레임 6 : 속도를 유지하세요.
프레임 7 : 속도를 유지하세요.
프레임 8 : 속도를 유지하세요.
프레임 9 : 속도를 유지하세요.
프레임 10 : 속도를 유지하세요.
프레임 11 : 속도를 유지하세요.
프레임 12 : 속도를 유지하세요.
프레임 13 : 속도를 유지하세요.
프레임 14 : 속도를 유지하세요.
프레임 15 : 속도를 유지하세요.
프레임 16 : 속도를 유지하세요.
프레임 17 : 속도를 유지하세요.
프레임 18 : 속도를 유지하세요.
프레임 19 : 속도를 유지하세요.
프레임 20 : 속도를 유지하세요.
프레임 21 : 속도를 유지하세요.
프레임 22 : 속도를 유지하세요.
프레임 23 : 속도를 유지하세요.
프레임 24 : 속도를 유지하세요.
프레임 25 : 속도를 유지하세요.
프레임 26 : 속도를 유지하세요.
프레임 27 : 속도를 유지하세요.
프레임 28 : 속도를 유지하세요.
프레임 29 : 속도를 유지하세요.
프레임 30 : 속도를 유지하세요.


KeyboardInterrupt: 